In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Mlib').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/18 15:26:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/18 15:26:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/18 15:26:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
training = spark.read.csv('test4.csv', header = True, inferSchema = True)

In [3]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [10]:
# Assemble independent features for Mlib 
#[Age, Experience] --> new feature --> independent feature

In [7]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols= ["age","Experience"], outputCol= "Independent")

In [8]:
output = featureassembler.transform(training)

In [9]:
output.show()

+---------+---+----------+------+-----------+
|     Name|age|Experience|Salary|Independent|
+---------+---+----------+------+-----------+
|    Krish| 31|        10| 30000|[31.0,10.0]|
|Sudhanshu| 30|         8| 25000| [30.0,8.0]|
|    Sunny| 29|         4| 20000| [29.0,4.0]|
|     Paul| 24|         3| 20000| [24.0,3.0]|
|   Harsha| 21|         1| 15000| [21.0,1.0]|
|  Shubham| 23|         2| 18000| [23.0,2.0]|
+---------+---+----------+------+-----------+



In [11]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent']

In [13]:
# Subset final colmns
finalized_data = output.select("Independent","Salary")

In [14]:
finalized_data.show()

+-----------+------+
|Independent|Salary|
+-----------+------+
|[31.0,10.0]| 30000|
| [30.0,8.0]| 25000|
| [29.0,4.0]| 20000|
| [24.0,3.0]| 20000|
| [21.0,1.0]| 15000|
| [23.0,2.0]| 18000|
+-----------+------+



In [59]:
#Splitting input into train-test  and fit linear regression model on train
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.60,0.40]) # Using randomSplit for splitting

In [60]:
train_data.show()

+-----------+------+
|Independent|Salary|
+-----------+------+
| [23.0,2.0]| 18000|
| [24.0,3.0]| 20000|
|[31.0,10.0]| 30000|
+-----------+------+



In [61]:
test_data.show()

+-----------+------+
|Independent|Salary|
+-----------+------+
| [21.0,1.0]| 15000|
| [29.0,4.0]| 20000|
| [30.0,8.0]| 25000|
+-----------+------+



In [62]:
type(finalized_data)

pyspark.sql.dataframe.DataFrame

In [63]:
regressor = LinearRegression(featuresCol='Independent', labelCol= 'Salary') # model definition

In [64]:
regressor = regressor.fit(train_data) # training using .fit method

23/08/18 15:47:10 WARN Instrumentation: [95b72ad9] regParam is zero, which might cause numerical instability and overfitting.
23/08/18 15:47:10 WARN Instrumentation: [95b72ad9] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [67]:
regressor.coefficients

DenseVector([736.8421, 736.8421])

In [68]:
regressor.intercept

-175.43859649151398

In [66]:
## Evaluate using test_data both Independent and feature colmns
pred_results = regressor.evaluate(test_data)

In [69]:
# Prediction
pred_results.predictions.show()

+-----------+------+------------------+
|Independent|Salary|        prediction|
+-----------+------+------------------+
| [21.0,1.0]| 15000|16035.087719298195|
| [29.0,4.0]| 20000| 24140.35087719302|
| [30.0,8.0]| 25000| 27824.56140350881|
+-----------+------+------------------+



In [71]:
## Model performance using mean absolute error
pred_results.meanAbsoluteError

2666.6666666666756